<a href="https://colab.research.google.com/github/farihahahmed/solar-forecasting-AI-novel-cyclical-features-encoding-technique/blob/main/1_Data_Transformation_of_Ausgrid_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to:

1. Transform the raw Ausgrid data, or **Dataset1.csv** into **Dataset2.csv** which will be imported into Excel later for some more transformations (to prepare out data for input and output matrices)


2. Transform the raw Ausgrid data, **Dataset1.csv**, into **Dataset3.csv** which will mainly just be used to plot the data and see/compare seasonal variations in the data

3. Plot **Dataset 3.csv** to explore seasonal variations

**1. Creating Dataset2.csv from Dataset1.csv**

In [ ]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Restarting Lumiere - Farihah and Mauricio/Notebooks and data/Dataset1.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Initial view of how the data looks

df.head()

,"2012-2013 Solar home electricity data - Before using this data, please read the attached pdf document ""Ausgrid solar home electricity data notes (Aug 2014).pdf""",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,3:00,3:30,4:00,4:30,5:00,5:30,6:00,6:30,7:00,7:30,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00,Row Quality
1,1,3.78,2076,CL,1/07/2012,1.25,1.25,1.25,1.263,0.131,0.569,0.556,0,0,0.95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.081,NaN
2,1,3.78,2076,GC,1/07/2012,0.855,0.786,0.604,0.544,0.597,0.612,1.245,0.665,0.076,0.14,0.657,0.337,0.048,0.101,0.05,0.219,0.334,0.164,0.374,0.201,0.077,0.38,0.051,0.11,0.043,0.109,0.292,0.109,0.042,0.107,0.043,0.409,0.133,0.099,0.087,0.087,0.385,0.211,0.329,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.09,0.068,NaN
3,1,3.78,2076,GG,1/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.006,0.038,0.088,0.175,0.281,0.419,0.581,0.619,0.681,0.488,0.513,0.125,0.95,1.156,1.056,0.7,0.6,0.2,0.044,0.006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,1,3.78,2076,CL,2/07/2012,1.25,1.25,1.125,0,0.925,0.231,0,0.488,0.513,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.069,NaN


In [ ]:
df.tail()

,"2012-2013 Solar home electricity data - Before using this data, please read the attached pdf document ""Ausgrid solar home electricity data notes (Aug 2014).pdf""",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
268553,300,3.36,2086,GC,29/06/2013,0.171,0.832,0.44,0.745,0.149,0.186,0.149,0.175,0.172,0.156,0.181,0.156,0.17,0.172,0.772,1.845,1.175,1.175,0.501,1.27,1.084,1.885,1.627,1.832,1.946,1.029,0.537,0.46,0.526,1.089,1.574,2.403,1.288,0.531,0.531,1.239,1.302,0.88,0.436,0.383,0.404,0.367,0.398,0.35,0.228,0.17,0.139,0.171,NaN
268554,300,3.36,2086,GG,29/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
268555,300,3.36,2086,CL,30/06/2013,2.501,2.475,2.491,2.508,2.49,2.502,0.339,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.468,NaN
268556,300,3.36,2086,GC,30/06/2013,0.15,0.158,0.157,0.137,0.163,0.135,0.167,0.15,0.148,0.167,0.138,0.166,0.138,0.157,0.145,0.348,0.197,0.227,1.914,1.258,1.197,2.207,1.999,1.441,1.355,0.264,0.137,0.783,1.151,1.208,0.497,0.548,1.278,2.963,2.426,2.54,2.641,1.405,0.97,0.625,0.595,0.695,0.646,0.581,0.458,0.472,0.732,0.297,NaN
268557,300,3.36,2086,GG,30/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
#Transforming the originl dataset

#Fixing the header of the dataframe
header_row = 0
df.columns = df.iloc[header_row]
df = df.drop(header_row)

#Resetting the index
df = df.reset_index(drop=True)
df.head()

#Deleting CL and GC data
df = df.rename({'Consumption Category':'type'}, axis='columns')
df = df[df.type != 'CL']
df = df[df.type != 'GC']

#Resetting index again
df = df.reset_index(drop=True)

#Renaming the columns we need
df = df.rename({'Customer':'customer_num', 'Generator Capacity':'gen_cap'}, axis='columns')

#Dropping unnecessary columns
df = df.drop("Postcode", axis=1)
df = df.drop("type", axis=1)
df = df.drop("Row Quality", axis=1)
df = df.drop("gen_cap", axis=1)

In [ ]:
#Seeing how our dataset looks

df.head()

,customer_num,date,0:30,1:00,1:30,2:00,2:30,3:00,3:30,4:00,4:30,5:00,5:30,6:00,6:30,7:00,7:30,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,1/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.006,0.038,0.088,0.175,0.281,0.419,0.581,0.619,0.681,0.488,0.513,0.125,0.95,1.156,1.056,0.7,0.6,0.2,0.044,0.006,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.019,0.025,0.044,0.244,0.481,0.663,0.769,0.844,1,0.775,0.756,0.869,0.725,0.744,0.913,0.631,0.294,0.031,0.006,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,3/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.019,0.031,0.05,0.213,0.481,0.663,0.769,0.838,0.919,1,1.031,1.1,1.138,1.006,0.838,0.688,0.35,0.069,0.013,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,4/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.019,0.044,0.081,0.413,0.556,0.75,0.763,0.831,0.944,1.031,0.925,0.588,0.681,0.7,0.219,0.494,0.256,0.044,0.006,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,5/07/2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.006,0.038,0.15,0.175,0.219,0.319,0.444,0.619,0.675,0.625,0.744,0.644,0.413,0.144,0.406,0.619,0.325,0.413,0.094,0.044,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.tail()

,customer_num,date,0:30,1:00,1:30,2:00,2:30,3:00,3:30,4:00,4:30,5:00,5:30,6:00,6:30,7:00,7:30,8:00,8:30,9:00,9:30,10:00,10:30,11:00,11:30,12:00,12:30,13:00,13:30,14:00,14:30,15:00,15:30,16:00,16:30,17:00,17:30,18:00,18:30,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
109414,300,26/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109415,300,27/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109416,300,28/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109417,300,29/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109418,300,30/06/2013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#checking for null values
#should be 109419 non-null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109419 entries, 0 to 109418
Data columns (total 50 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_num  109419 non-null  object
 1   date          109419 non-null  object
 2   0:30          109419 non-null  object
 3   1:00          109419 non-null  object
 4   1:30          109419 non-null  object
 5   2:00          109419 non-null  object
 6   2:30          109419 non-null  object
 7   3:00          109419 non-null  object
 8   3:30          109419 non-null  object
 9   4:00          109419 non-null  object
 10  4:30          109419 non-null  object
 11  5:00          109419 non-null  object
 12  5:30          109419 non-null  object
 13  6:00          109419 non-null  object
 14  6:30          109419 non-null  object
 15  7:00          109419 non-null  object
 16  7:30          109419 non-null  object
 17  8:00          109419 non-null  object
 18  8:30          109419 non

In [ ]:
from google.colab import files
df.to_csv('Dataset2.csv')
files.download('Dataset2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**2. Creating Dataset3.csv from Dataset1.csv**

In [ ]:
#Saving all values from the original dataset into smaller variables
#This will help us form our new dataset, which will be shaped slightly different
customer_num = df['customer_num']
date = df['date']
pv_030 = df['0:30']
pv_100 = df['1:00']
pv_130 = df['1:30']
pv_200 = df['2:00']
pv_230 = df['2:30']
pv_300 = df['3:00']
pv_330 = df['3:30']
pv_400 = df['4:00']
pv_430 = df['4:30']
pv_500 = df['5:00']
pv_530 = df['5:30']
pv_600 = df['6:00']
pv_630 = df['6:30']
pv_700 = df['7:00']
pv_730 = df['7:30']
pv_800 = df['8:00']
pv_830 = df['8:30']
pv_900 = df['9:00']
pv_930 = df['9:30']
pv_1000 = df['10:00']
pv_1030 = df['10:30']
pv_1100 = df['11:00']
pv_1130 = df['11:30']
pv_1200 = df['12:00']
pv_1230 = df['12:30']
pv_1300 = df['13:00']
pv_1330 = df['13:30']
pv_1400 = df['14:00']
pv_1430 = df['14:30']
pv_1500 = df['15:00']
pv_1530 = df['15:30']
pv_1600 = df['16:00']
pv_1630 = df['16:30']
pv_1700 = df['17:00']
pv_1730 = df['17:30']
pv_1800 = df['18:00']
pv_1830 = df['18:30']
pv_1900 = df['19:00']
pv_1930 = df['19:30']
pv_2000 = df['20:00']
pv_2030 = df['20:30']
pv_2100 = df['21:00']
pv_2130 = df['21:30']
pv_2200 = df['22:00']
pv_2230 = df['22:30']
pv_2300 = df['23:00']
pv_2330 = df['23:30']
pv_000 = df['0:00']

In [ ]:
#Creating a new dataframe
new_df = pd.DataFrame()

In [ ]:
#Finding the amount of rows in our orignal dataset
length_df = df.shape[0]
print(length_df)

109419


In [ ]:
#Creating two lists, date_time and customer_numbers, which will be crucial parts of the new dataset

date_time = []
customer_numbers = []

#Multiplying the current number of rows by 48 (48 values per customer per day)
for i in range(length_df):
  for a in range(48):
    date_time.append(date[i])
    customer_numbers.append(customer_num[i])

In [ ]:
#Adding these two lists to my new df as key columns
new_df['customer_numbers'] = customer_numbers
new_df['date'] = date_time

In [ ]:
#Seeing how the new df looks
new_df.head()

,customer_numbers,date
0,1,1/07/2012
1,1,1/07/2012
2,1,1/07/2012
3,1,1/07/2012
4,1,1/07/2012


In [ ]:
#Adding a time column to our new_df, making sure that 48 time values are entered in for each row in the previous dataset (which was 109,419 rows)

time = ['0:30', '1:00', '1:30', '2:00', '2:30', '3:00', '3:30', '4:00', '4:30', '5:00', '5:30', '6:00', '6:30',
        '7:00', '7:30', '8:00', '8:30', '9:00', '9:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30',
        '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30',
        '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30', '23:00', '23:30', '0:00']

time = time*109419

In [ ]:
len(time)

5252112

In [ ]:
new_df['time'] = time

In [ ]:
#From the old df, transfering all of the PV power values into a new list for power
#Before, all the power values were lined horizontally, now they are lined vertically

power = []

for i in range(len(customer_num)):
  power.append(pv_030[i])
  power.append(pv_100[i])
  power.append(pv_130[i])
  power.append(pv_200[i])
  power.append(pv_230[i])
  power.append(pv_300[i])
  power.append(pv_330[i])
  power.append(pv_400[i])
  power.append(pv_430[i])
  power.append(pv_500[i])
  power.append(pv_530[i])
  power.append(pv_600[i])
  power.append(pv_630[i])
  power.append(pv_700[i])
  power.append(pv_730[i])
  power.append(pv_800[i])
  power.append(pv_830[i])
  power.append(pv_900[i])
  power.append(pv_930[i])
  power.append(pv_1000[i])
  power.append(pv_1030[i])
  power.append(pv_1100[i])
  power.append(pv_1130[i])
  power.append(pv_1200[i])
  power.append(pv_1230[i])
  power.append(pv_1300[i])
  power.append(pv_1330[i])
  power.append(pv_1400[i])
  power.append(pv_1430[i])
  power.append(pv_1500[i])
  power.append(pv_1530[i])
  power.append(pv_1600[i])
  power.append(pv_1630[i])
  power.append(pv_1700[i])
  power.append(pv_1730[i])
  power.append(pv_1800[i])
  power.append(pv_1830[i])
  power.append(pv_1900[i])
  power.append(pv_1930[i])
  power.append(pv_2000[i])
  power.append(pv_2030[i])
  power.append(pv_2100[i])
  power.append(pv_2130[i])
  power.append(pv_2200[i])
  power.append(pv_2230[i])
  power.append(pv_2300[i])
  power.append(pv_2330[i])
  power.append(pv_000[i])

In [ ]:
#Adding this list, power, to the new_df

new_df["PV_power"] = power

In [ ]:
new_df["datetime"] = new_df["date"] + " " + new_df["time"]

In [ ]:
new_df["datetime"] = pd.to_datetime(new_df["datetime"], format='%d/%m/%Y %H:%M')

In [ ]:
new_df = new_df.drop(["date", "time"], axis=1)

In [ ]:
new_df

,customer_numbers,PV_power,datetime
0,1,0,2012-07-01 00:30:00
1,1,0,2012-07-01 01:00:00
2,1,0,2012-07-01 01:30:00
3,1,0,2012-07-01 02:00:00
4,1,0,2012-07-01 02:30:00
...,...,...,...
5252107,300,0,2013-06-30 22:00:00
5252108,300,0,2013-06-30 22:30:00
5252109,300,0,2013-06-30 23:00:00
5252110,300,0,2013-06-30 23:30:00


In [ ]:
from google.colab import files
new_df.to_csv('Dataset3.csv')
files.download('Dataset3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>